## 6.4 循环神经网络的从零开始实现

In [20]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append('..')
import d2lzh_pytorch.utils as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

### 6.4.1 one-hot向量

> 1. **假设词典中不同字符的数量为N,每个字符已经同0-N-1的连续整数值索引一一对应**
> 2. **假设某个字符索引为i,创建一个全0的长为N的向量,且位置i的元素设置为1**

In [24]:
def one_hot(x, n_class, dtype=torch.float32):
    # X shape: (batch), output_shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    # scatter_(input, dim, index, src) 
    # 将src中的所有值按照index确定的索引写入本tensor中。其中索引是根据给定的dimension，dim按照gather()描述的规则来确定
    # 此时dim=1
    res.scatter_(1, x.view(-1, 1), 1)
    return res

x = torch.tensor([0, 2])
y = F.one_hot(x, vocab_size)
print(x.shape[0])
one_hot(x, vocab_size)

2


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

[scatter_()中文文档](https://pytorch-cn.readthedocs.io/zh/latest/package_references/Tensor/#scatter_input-dim-index-src-tensor)

In [25]:
a = torch.arange(10, dtype=torch.float32).view(2, 5)
print(a)
b = torch.zeros(3, 5).scatter_(0, torch.LongTensor([[0, 1, 2, 0, 0], [2, 0, 0, 1, 2]]), a)
print(b)

# 解释, 记tmp = [[0, 1, 2, 0, 0], [2, 0, 0, 1, 2]], dim=0, 对列进行填充
# tmp[0][0] = 0 -> a中取a[0][0] = 0, 将其插入到第0列,第0行位置,b[0][0] = 0
# tmp[0][1] = 1 -> a中取a[0][1] = 1, 将其插入到第1列,第1行位置,b[1][1] = 1
# tmp[0][2] = 2 -> a中取a[0][2] = 2，将其插入到第2列,第2行位置,b[2][2] = 2
# ...
# tmp[1][0] = 2 -> a中取a[1][0] = 5，将其插入到第0列,第2行位置,b[2][0] = 5
# tmp[1][1] = 0 -> a中取a[1][1] = 6，将其插入到第1列,第1行位置,b[0][1] = 6
# ...

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])
tensor([[0., 6., 7., 3., 4.],
        [0., 1., 0., 8., 0.],
        [5., 0., 2., 0., 9.]])


> 1. **采样的小批量形状为(批量大小, 时间步数)**
> 2. **可将这样的小批量转为数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵**
> 3. **矩阵个数=时间步数,即时间步t的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$**;其中n为批量大小,d为输入个数,即one-hot向量长度

In [26]:
def to_onehot(X, n_class):
    # X shape: (batch, seq_len), out_put: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs))
print(inputs[0].shape)

5
torch.Size([2, 1027])


### 6.3.2 初始化模型参数

In [27]:
# num_hiddens 是隐藏单元个数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])


### 6.4.3 定义模型

In [28]:
# 初始化隐藏状态, 返回元祖
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [29]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [30]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


### 6.4.4 定义预测函数

In [31]:
# prefix(含有数个字符的字符串)来预测接下来的num_chars个字符
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [32]:
# 用前缀为'分开'造歌词
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
           device, idx_to_char, char_to_idx)

'分开伦福恍写若托依甘作沙'

### 6.4.5 裁剪梯度

> 1. **`RNN`容易出现梯度衰减或者梯度爆炸,可通过`裁剪梯度(clip gradient)`**
> 2. **假设把所有模型参数梯度的元素凭拼接成一个向量$\boldsymbol{g}$,并设置裁剪的阈值,裁剪后的梯度**
$$\min\left(\frac{\theta}{||\boldsymbol{g}||}, 1\right)\boldsymbol{g}$$
**的$\boldsymbol{L}_2$不超过$\theta$**

In [38]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

### 6.4.6 困惑度

> 1. **通常用`困惑度(preplexity)`评价语言模型的好坏;困惑度是对交叉熵做指数运算后得到的值**
    - **最佳情况: 模型总是把标签类别的概率预测为1,此时困惑度为1**
    - **最差情况下: 模型总是把标签类别的概率预测为0,此时困惑度为正无穷**
    - **基线情况下: 模型总是预测所有类别的概率都相同,此时困惑度为类别个数**

### 6.4.6 定义模型训练函数

> 1. **使用困惑度评价模型**
> 2. **在迭代模型参数前裁剪梯度**
> 3. **对时序数据采用不同采样方法将导致隐藏状态初始化的不同**

In [39]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                         vocab_size, device, corpus_indices, idx_to_char,
                         char_to_idx, is_random_iter, num_epochs, num_steps,
                         lr, clipping_theta, batch_size, pred_period,
                         pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter: # 如使用相邻采样,在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter: # 如果采用随机采样,在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:
                # 否则需要使用detach函数从计算图分离隐藏状态,主要是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后的形状为(num_steps, batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps), 转置后再变成为长度为
            # batch * num_steps的向量, 这样根输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device) # 裁剪梯度
            d2l.sgd(params, lr, 1) # 因为误差已经取过平均值,梯度不用在做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, preplexity %f, time %.2f sec' % (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                                       num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

### 6.4.8 训练模型并创作歌词

In [40]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [41]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, preplexity 68.751274, time 0.73 sec
 - 分开 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 爱要的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱
epoch 100, preplexity 10.555276, time 0.75 sec
 - 分开 一颗两 一颗两人 在人忆 的片段 三间些 干什么 三颗两 三直两人三江四等 连成线背著比我妈妈 我
 - 不分开 我不多 你怎么 我想就这样牵着 干什么 干什么 我想就这样牵着你的手不放开 爱能不能够永远单纯没有
epoch 150, preplexity 2.931636, time 0.69 sec
 - 分开 爱什么不多 有慢都红天 到子在角前 不爽开反走 有不在角走 这不在角走 有不横 说 懂的烟里 没有
 - 不分开扫 我后你有多 让我知着你 娘子 娘子却多战不  却穿了其实我的愿望就怎么小 就怎么每天祈 我的念 
epoch 200, preplexity 1.603930, time 0.72 sec
 - 分开 装什么不多 有话苦苦 全家怕日出 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋
 - 不分开扫简的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在这了过  让笑梦夫吃 一杰就反
epoch 250, preplexity 1.333466, time 0.70 sec
 - 分开 爱子心不过个雕花啦平 脑养的黑猫笑起来像哭 啦啦啦呜 在根我不抽的菸 在成风向了始的誓言 纪录第一
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在知了不屋 白的蜡烛 温暖了空屋
